# DataSens E1_v1 — 02_schema_create

- Objectifs: créer le schéma (SQLite), 10 tables Pareto
- Prérequis: exécuter 01_setup_env
- Sortie: base `datasens/datasens.db`
- Guide: docs/GUIDE_TECHNIQUE_E1.md



> Notes:
> - SQLite (fichier local) simplifie la démo E1_v1.
> - `sqlite3.connect(DB_PATH)` ouvre (ou crée) la base.
> - `CREATE TABLE IF NOT EXISTS` évite les erreurs si la table existe déjà.
> - `conn.commit()` enregistre les changements.
> - Une ligne est ajoutée dans `README_VERSIONNING.md` pour la traçabilité.
> - Les tables couvrent les entités principales collecte/gouvernance.


In [ ]:
# DataSens E1_v1 - 02_schema_create
# 💾 Base SQLite et schéma (18 tables)
import sqlite3
from datetime import datetime
from pathlib import Path

# Détection robuste du dossier projet (remonte jusqu'à trouver le dossier avec notebooks/ et docs/)
current = Path.cwd()
PROJECT_ROOT = None
# Remonter jusqu'à trouver un dossier qui contient à la fois notebooks/ et docs/
while current != current.parent:
    if (current / "notebooks").exists() and (current / "docs").exists():
        PROJECT_ROOT = current
        break
    current = current.parent
else:
    # Fallback: utilise le répertoire courant si pas trouvé
    PROJECT_ROOT = Path.cwd()

# Emplacement du fichier SQLite (BDD locale de démo E1_v1)
DB_PATH = PROJECT_ROOT / "datasens" / "datasens.db"
print(f"📂 Racine projet détectée : {PROJECT_ROOT}")
DB_PATH.parent.mkdir(parents=True, exist_ok=True)

# Connexion à la base SQLite (créée si absente)
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()
print(f"✅ Base SQLite prête : {DB_PATH}")

# Tables principales (collecte/gouvernance) inspirées du monolithe
TABLES_SQL = {
    # Typologie des données (RSS, API, Scraping, CSV, Big Data)
    "type_donnee": """
    CREATE TABLE IF NOT EXISTS type_donnee (
        id_type_donnee INTEGER PRIMARY KEY AUTOINCREMENT,
        libelle TEXT, description TEXT,
        frequence_maj TEXT, categorie_metier TEXT
    );""",
    # Source (provenance) rattachée à un type de donnée
    "source": """
    CREATE TABLE IF NOT EXISTS source (
        id_source INTEGER PRIMARY KEY AUTOINCREMENT,
        id_type_donnee INTEGER,
        nom TEXT, url TEXT, fiabilite REAL,
        FOREIGN KEY (id_type_donnee) REFERENCES type_donnee(id_type_donnee)
    );""",
    # Flux (opération de collecte datée + format + manifest)
    "flux": """
    CREATE TABLE IF NOT EXISTS flux (
        id_flux INTEGER PRIMARY KEY AUTOINCREMENT,
        id_source INTEGER, date_collecte TEXT, format TEXT, manifest_uri TEXT,
        FOREIGN KEY (id_source) REFERENCES source(id_source)
    );""",
    # Document (contenu final) rattaché à un flux
    "document": """
    CREATE TABLE IF NOT EXISTS document (
        id_doc INTEGER PRIMARY KEY AUTOINCREMENT,
        id_flux INTEGER, titre TEXT, texte TEXT, langue TEXT, date_publication TEXT,
        FOREIGN KEY (id_flux) REFERENCES flux(id_flux)
    );""",
    # Référentiels territoriaux et indicateurs (simplifiés E1)
    "pays": "CREATE TABLE IF NOT EXISTS pays (id_pays INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT);",
    "region": "CREATE TABLE IF NOT EXISTS region (id_region INTEGER PRIMARY KEY AUTOINCREMENT, id_pays INTEGER, nom TEXT, FOREIGN KEY (id_pays) REFERENCES pays(id_pays));",
    "departement": "CREATE TABLE IF NOT EXISTS departement (id_departement INTEGER PRIMARY KEY AUTOINCREMENT, id_region INTEGER, code_dept TEXT, nom TEXT, FOREIGN KEY (id_region) REFERENCES region(id_region));",
    "commune": "CREATE TABLE IF NOT EXISTS commune (id_commune INTEGER PRIMARY KEY AUTOINCREMENT, id_departement INTEGER, code_insee TEXT, nom_commune TEXT, lat REAL, lon REAL, population INTEGER, FOREIGN KEY (id_departement) REFERENCES departement(id_departement));",
    "territoire": "CREATE TABLE IF NOT EXISTS territoire (id_territoire INTEGER PRIMARY KEY AUTOINCREMENT, id_commune INTEGER, FOREIGN KEY (id_commune) REFERENCES commune(id_commune));",
    "type_indicateur": "CREATE TABLE IF NOT EXISTS type_indicateur (id_type_indic INTEGER PRIMARY KEY AUTOINCREMENT, code TEXT, libelle TEXT, unite TEXT);",
    "indicateur": "CREATE TABLE IF NOT EXISTS indicateur (id_indic INTEGER PRIMARY KEY AUTOINCREMENT, id_territoire INTEGER, id_type_indic INTEGER, valeur REAL, annee INTEGER, FOREIGN KEY (id_territoire) REFERENCES territoire(id_territoire), FOREIGN KEY (id_type_indic) REFERENCES type_indicateur(id_type_indic));",
    "meteo": "CREATE TABLE IF NOT EXISTS meteo (id_meteo INTEGER PRIMARY KEY AUTOINCREMENT, id_territoire INTEGER, date_obs TEXT, temperature REAL, humidite REAL, vent_kmh REAL, pression REAL, FOREIGN KEY (id_territoire) REFERENCES territoire(id_territoire));",
    "utilisateur": "CREATE TABLE IF NOT EXISTS utilisateur (id_user INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT, role TEXT, organisation TEXT);",
    "emotion": "CREATE TABLE IF NOT EXISTS emotion (id_emotion INTEGER PRIMARY KEY AUTOINCREMENT, type_emotion TEXT, valence TEXT, score_confiance REAL);",
    "annotation": "CREATE TABLE IF NOT EXISTS annotation (id_annotation INTEGER PRIMARY KEY AUTOINCREMENT, id_doc INTEGER, id_user INTEGER, id_emotion INTEGER, intensity REAL, polarity TEXT, date_annotation TEXT, FOREIGN KEY (id_doc) REFERENCES document(id_doc), FOREIGN KEY (id_user) REFERENCES utilisateur(id_user), FOREIGN KEY (id_emotion) REFERENCES emotion(id_emotion));",
    "theme": "CREATE TABLE IF NOT EXISTS theme (id_theme INTEGER PRIMARY KEY AUTOINCREMENT, libelle TEXT, description TEXT);",
    "evenement": "CREATE TABLE IF NOT EXISTS evenement (id_event INTEGER PRIMARY KEY AUTOINCREMENT, id_theme INTEGER, date_event TEXT, avg_tone REAL, source_event TEXT, FOREIGN KEY (id_theme) REFERENCES theme(id_theme));",
    "document_evenement": "CREATE TABLE IF NOT EXISTS document_evenement (id_doc INTEGER, id_event INTEGER, PRIMARY KEY (id_doc, id_event), FOREIGN KEY (id_doc) REFERENCES document(id_doc), FOREIGN KEY (id_event) REFERENCES evenement(id_event));"
}

# Exécution séquentielle des DDL
tables_created = []
for name, sql in TABLES_SQL.items():
    cur.execute(sql)
    tables_created.append(name)
    print(f"✅ Table : {name}")
conn.commit()

# 📊 Visualisation : Graphique des tables créées par catégorie
import matplotlib.pyplot as plt
import pandas as pd

# Catégorisation des tables
categories = {
    "Collecte & Traçabilité": ["type_donnee", "source", "flux", "document"],
    "Géographie": ["pays", "region", "departement", "commune", "territoire"],
    "Données Métier": ["meteo", "indicateur", "type_indicateur", "evenement", "theme"],
    "Annotation & IA": ["utilisateur", "emotion", "annotation", "document_evenement"]
}

# Compter les tables par catégorie
cat_counts = {cat: len([t for t in tables_created if t in cat_list]) 
              for cat, cat_list in categories.items()}

# Créer le graphique
df_tables = pd.DataFrame(list(cat_counts.items()), columns=["Catégorie", "Nombre"])
df_tables = df_tables.sort_values("Nombre", ascending=False)

plt.figure(figsize=(10, 6))
colors = plt.cm.Pastel1(range(len(df_tables)))
bars = plt.bar(df_tables["Catégorie"], df_tables["Nombre"], color=colors)

# Ajouter les valeurs sur les barres
for bar, value in zip(bars, df_tables["Nombre"]):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             str(value), ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.title("📊 Schéma créé : Répartition des 18 tables par catégorie", 
          fontsize=14, fontweight='bold')
plt.xlabel("Catégorie", fontsize=12)
plt.ylabel("Nombre de tables", fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis="y", linestyle="--", alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📈 Total : {len(tables_created)} tables créées")
for cat, count in cat_counts.items():
    print(f"   • {cat} : {count} tables")

# 📋 Tables de données réelles : Afficher la structure de chaque table créée
print("\n📋 Structure des tables créées (schéma SQLite) :")
df_schema = pd.DataFrame({
    'Table': tables_created,
    'Statut': ['Créée' for _ in tables_created]
})
display(df_schema)

# Afficher les informations détaillées des tables principales
print("\n🔍 Détails des tables principales (colonnes) :")
for table_name in ["type_donnee", "source", "flux", "document"]:
    if table_name in tables_created:
        try:
            # Récupérer la structure de la table
            schema_info = cur.execute(f"PRAGMA table_info({table_name})").fetchall()
            if schema_info:
                df_cols = pd.DataFrame(schema_info, columns=['cid', 'name', 'type', 'notnull', 'default_value', 'pk'])
                df_cols = df_cols[['name', 'type', 'pk', 'notnull']]
                df_cols.columns = ['Colonne', 'Type', 'PK', 'NOT NULL']
                print(f"\n   📊 Table '{table_name}' :")
                display(df_cols)
        except Exception as e:
            pass

# Journalisation (traçabilité pour le jury)
with open(PROJECT_ROOT / "README_VERSIONNING.md", "a", encoding="utf-8") as f:
    f.write(f"- **{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}** | `DB_SCHEMA` | Création des 18 tables principales\n")
print("\n✅ Schéma créé et visualisé.")


📂 Racine projet détectée : c:\Users\Utilisateur\Desktop\DataSens\notebooks\datasens_E1_v1
✅ Base SQLite prête : c:\Users\Utilisateur\Desktop\DataSens\notebooks\datasens_E1_v1\datasens\datasens.db
✅ Table : type_donnee
✅ Table : source
✅ Table : flux
✅ Table : document
✅ Table : pays
✅ Table : region
✅ Table : departement
✅ Table : commune
✅ Table : territoire
✅ Table : type_indicateur
✅ Table : indicateur
✅ Table : meteo
✅ Table : utilisateur
✅ Table : emotion
✅ Table : annotation
✅ Table : theme
✅ Table : evenement
✅ Table : document_evenement
Schéma créé.
